In [5]:
from pathlib import Path
import numpy as np

In [112]:
def read(prefix='data', suffix='23', pad_width=None):
    
    lines = Path(f'{prefix}/{suffix}.txt').read_text().rstrip().split('\n')
    M = [[(c == '#') for c in l] for l in lines]
    if pad_width is None:
        pad_width = len(lines)

    return np.pad(np.array(M), pad_width=pad_width)

def disp(M):
    print(*[''.join('#' if c else '.' for c in r) for r in M], sep='\n')

In [82]:
def check(M, y,x, dy, dx):
    assert abs(dx + dy) == 1
    yr = slice(y-1,y+2) if dy == 0 else y+dy
    xr = slice(x-1,x+2) if dx == 0 else x+dx
    return M[yr,xr].sum() == 0


In [142]:


def propose(M, y, x, dirs):
    assert y > 0 and y < M.shape[0] -1
    assert x > 0 and x < M.shape[1] -1

    if M[y-1:y+2,x-1:x+2].sum() == 1:
        return None
    for (dy,dx) in dirs:
        if check(M, y,x, dy, dx):
            return (y+dy, x+dx)
    return None

In [197]:
from collections import Counter
def round(M, dirs):
    proposed = Counter()
    moves = []
    moved = False
    for (y,x) in zip(*np.where(M)):
        to = propose(M, y, x, dirs)
        if to is not None:
            proposed[to] += 1
            moves.append(((y,x),(to)))

    for (fr, to) in moves:
        if proposed[to] == 1:
            M[*to] = True
            M[*fr] = False
            moved = True
    return moved

In [211]:
original_dirs = [(-1,0),(1,0),(0,-1),(0,1)]
def simulate(M, num_rounds, verbose=False):
    M = M.copy()
    dirs = original_dirs
    for idx in range(num_rounds):
        moved = round(M, dirs)
        if not moved:
            return (M, idx+1)
        if verbose:
            print(dirs)
            print()
            disp(M)
            print()
        dirs = dirs[1:] + [dirs[0]]
    return (M, idx+1)

In [212]:
def empty(M):
    W = np.where(M)
    y_min, y_max = W[0].min(), W[0].max()
    x_min, x_max = W[1].min(), W[1].max()
    return (~M[y_min:y_max+1, x_min:x_max+1]).sum()

In [213]:
M = read('test', '23-2')
M, idx = simulate(M,3, verbose=False)
disp(M)

.................
.................
.................
.................
.................
.................
........#........
..........#......
......#..........
..........#......
.................
........#........
.................
.................
.................
.................
.................
.................


In [214]:
M = read('test')
M, idx = simulate(M,10, verbose=False)
disp(M)
empty(M), idx

.....................
.....................
.....................
.....................
.....................
...........#.........
...............#.....
......#.#..#.........
..........#..........
.......#.....#..#....
.....#......##.......
.........##..........
......#........#.....
........#.#..#.......
.....................
........#..#..#......
.....................
.....................
.....................
.....................
.....................


(110, 10)

In [215]:
M = read()
M,idx = simulate(M,10, verbose=False)
empty(M)

4162

In [217]:
M = read('test')
M, idx = simulate(M,100, verbose=False)
empty(M), idx

(146, 20)

In [218]:
M = read()
M, idx = simulate(M,1000, verbose=False)
empty(M), idx

(17210, 986)